In [95]:
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.common.keys import Keys as keys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime
from datetime import date,timedelta

def url_Time(url,year,month,day):
    now_date = date(year,month,day)
    old_date = now_date-timedelta(days=8030)
    URL_0 = url.split('from=')[0]
    URL_1 = url.split('from=')[1]
    URL_1_1 = URL_1.split('&to=')[1]
    URL_1_2 = URL_1_1.split('&time')[1]
    URL = URL_0+'from='+str(old_date)+'&to='+str(now_date)+'&time'+URL_1_2
    return URL

URL = url_Time('https://finance.yahoo.co.jp/quote/4661.T/history?from=20201105&to=20211105&timeFrame=d&page=1',2021,11,10)
URL_1 = re.sub('-','',URL)


browser = webdriver.Chrome()
browser.get(URL_1)

j = 0
i = 0
Data_list = []
while i < 245:
    tag_ = browser.find_elements_by_class_name("_2ZqX1qip")
    tag_len = len(tag_)
    for j in range(tag_len):
        tag_text = tag_[j].text
        tag_Data = tag_text.split('\n')
        Data_list.append(tag_Data)
        j += 1
    name_ = browser.find_element_by_tag_name("thead")
    name_Data = name_.text.split(' ')
    df_ = pd.DataFrame(data=Data_list,columns=name_Data)
    next_ = browser.find_elements_by_class_name("cVr3KGRY")
    next_[1].click()
    i += 1
    time.sleep(1)
    
df_list_start = []
df_list_max = []
df_list_mine = []
df_list_end = []
df_list_volume = []
df_list_change = []
df_list_day = []
i = 0
for i in range(len(df_2)):
    try:
        df_day = (df_.loc[i]['日付'])
        df_start = (df_.loc[i]['始値'])
        df_max = (df_.loc[i]['高値'])
        df_mine = (df_.loc[i]['安値'])
        df_end = (df_.loc[i]['終値'])
        df_volume = (df_.loc[i]['出来高'])
        df_start_1 = re.sub(',','',df_start)
        df_max_1 = re.sub(',','',df_max)
        df_mine_1 = re.sub(',','',df_mine)
        df_end_1= re.sub(',','',df_end)
        df_volume_1 = re.sub(',','',df_volume)
    except TypeError:
        continue
    finally:
        df_list_start.append(df_start_1)
        df_list_max.append(df_max_1)
        df_list_mine.append(df_mine_1)
        df_list_end.append(df_end_1)
        df_list_volume.append(df_volume_1)
        df_list_day.append(df_day)
        df_2 = pd.DataFrame(data = df_list_day,columns=['日付'])
        df_2['始値'] = df_list_start
        df_2['高値'] = df_list_max
        df_2['安値'] = df_list_mine
        df_2['終値'] = df_list_end
        df_2['出来高'] = df_list_volume
        
b_list = []
for i in range(1,len(df_2)):
    a = df_list_end[i]
    b = df_list_end[i-1]
    z = (int(b)-int(a))*1/100
    b_list.append(z)
    if i == (len(df_2)-1):
        mean = sum(b_list)/len(b_list)
        Mean = round(mean,2)
        b_list.append(Mean)
        df_2['変化率'] = b_list

In [175]:
df_2 = df_2.drop(1616,axis=0)

In [101]:
df_2['出来高'] = df_2['出来高'].astype(np.int)

In [106]:
df_2['高値'] = df_2['高値'].astype(np.int)

In [107]:
df_2['安値'] = df_2['安値'].astype(np.int)

In [108]:
df_2['終値'] = df_2['終値'].astype(np.int)

In [176]:
df_2['始値'] = df_2['始値'].astype(np.int)

In [181]:
#単回帰分析による予測
#中心化
df_2_c = df_2 - df_2.mean()
#入力変数=x目的変数=y
x = df_2_c['始値']
y = df_2_c['終値']
#評価関数a
xx = x*x
xy = x*y
a = xy.sum()/xx.sum()
#予測関数化
def new_value(Value):
    x_new = Value
    mean = df_2.mean()
    xc = x_new - mean['始値']
    yc = a * xc
    #予測値をy_hat
    y_hat = yc + mean['終値']
    return y_hat

In [186]:
new_value(18700)

18718.468166206265